# This notebook is used to create a movie database in Neo4j Aura database
## Pre-requisite : Create a Neo4j Aura instance and keep it running

## Step 1 : upload movies.csv
## Step 2 : upload Persons.csv
## Step 3: upload Directed.csv
## Step 4: Upload Acted_in.csv

In [2]:
!pip install neo4j -q

In [4]:
#uri = "neo4j+s://your-instance-id.databases.neo4j.io"
#uri="neo4j+s://05d774a7.databases.neo4j.io"
uri="neo4j+s://8e403408.databases.neo4j.io"

In [3]:
uri="neo4j+s://8e403408.databases.neo4j.io"  # Get from Neo4j Aura console
username = "neo4j"  # usually 'neo4j'
password = "adswC-DiHRpM6BAnMbyHjoIIqEyrZUS2VsKcJdJ7LY4"  # set during DB setup


In [21]:
# Create the driver
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(username, password))

In [10]:
def read_nodes(driver):
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN n LIMIT 5")
        for record in result:
            node = record["n"]
            print(dict(node))

read_nodes(driver)


{'revenue': '373554033.0', 'languages': 'English', 'plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.", 'year': '1995', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'movieId': '1', 'title': 'Toy Story', 'released': '1995-11-22'}
{'revenue': '262797249.0', 'languages': 'English|French', 'plot': 'When two kids find and play a magical board game, they release a man trapped for decades in it and a host of dangers that can only be stopped by finishing the game.', 'year': '1995', 'genres': 'Adventure|Children|Fantasy', 'movieId': '2', 'title': 'Jumanji', 'released': '1995-12-15'}
{'revenue': '', 'languages': 'English', 'plot': "John and Max resolve to save their beloved bait shop from turning into an Italian restaurant, just as its new female owner catches Max's attention.", 'year': '1995', 'genres': 'Comedy|Romance', 'movieId': '3', 'title': 'Grumpier Old Men', 'released': '1995-12-22'}
{'revenue': '81

In [11]:
results = driver.session().run("MATCH (n) RETURN n LIMIT 5").data()
print(results)

[{'n': {'revenue': '373554033.0', 'languages': 'English', 'plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.", 'year': '1995', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'movieId': '1', 'title': 'Toy Story', 'released': '1995-11-22'}}, {'n': {'revenue': '262797249.0', 'languages': 'English|French', 'plot': 'When two kids find and play a magical board game, they release a man trapped for decades in it and a host of dangers that can only be stopped by finishing the game.', 'year': '1995', 'genres': 'Adventure|Children|Fantasy', 'movieId': '2', 'title': 'Jumanji', 'released': '1995-12-15'}}, {'n': {'revenue': '', 'languages': 'English', 'plot': "John and Max resolve to save their beloved bait shop from turning into an Italian restaurant, just as its new female owner catches Max's attention.", 'year': '1995', 'genres': 'Comedy|Romance', 'movieId': '3', 'title': 'Grumpier Old Men', 'released': '1995-

In [14]:
#read csv and upload to Neo4j Aura
from neo4j import GraphDatabase
import csv

def load_csv_to_neo4j(tx, row):
  #tx.run("MERGE (m:Movie {movieId: $movieId, title: $title,languages: $languages,plot: $plot,released: $released,revenue: $revenue,year: $year,genres:$genres})", movieId=row['movieId'], title=row['title'], languages=row['languages'],plot=row['plot'] ,released=row['released'] ,revenue=row['revenue'] ,year=row['year'],genres=row['genres'])
  #tx.run("MERGE (p:Person {person_tmdbId:$person_tmdbId, name: $name, dateofbirth: $born, birthcity:$bornIn})",person_tmdbId=row['person_tmdbId'], name=row['name'], born=row['born'], bornIn=row['bornIn'])
  #print(row)
  tx.run("MERGE (r:Ratings {movieId: $movieId, userId: $userId, name:$name,rating:$rating,timestamp:$timestamp})",movieId=row['\ufeffmovieId'], userId=row['userId'], name=row['name'],rating=row['rating'], timestamp=row['timestamp'])



In [15]:
with driver.session() as session:
    with open("ratings.csv", newline='') as csvfile: # Changed to movies.csv

        reader = csv.DictReader(csvfile)
        for row in reader:
            session.execute_write(load_csv_to_neo4j, row)

In [17]:
# --- Function to create ACTED_IN relationship ---
def create_acted_in(tx, person_tmdbId, movieId, role):
    tx.run(
        """
        MATCH (p:Person {person_tmdbId: $person_tmdbId})
        MATCH (m:Movie {movieId: $movieId})
        MERGE (p)-[r:ACTED_IN]->(m)
        ON CREATE SET r.role = $role
        """,
        person_tmdbId=person_tmdbId,
        movieId=movieId,
        role=role
    )

In [18]:
# --- Load CSV and create relationships ---
with driver.session() as session:
    with open("acted_in.csv", newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            #print(row)
            session.execute_write(
                create_acted_in,
                row['person_tmdbId'],
                row['\ufeffmovieId'],
                row.get('role', '')
            )

print("Relationships created successfully.")
driver.close()

Relationships created successfully.


In [19]:
# Function to create DIRECTED relationship
def create_directed(tx, person_tmdbId, movieId):
    tx.run(
        """
        MATCH (p:Person {person_tmdbId: $person_tmdbId})
        MATCH (m:Movie {movieId: $movieId})
        MERGE (p)-[:DIRECTED]->(m)
        """,
        person_tmdbId=person_tmdbId,
        movieId=movieId
    )

In [22]:
# --- Load CSV and create relationships ---
with driver.session() as session:
    with open("directed.csv", newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            #print(row)
            session.execute_write(
                create_directed,
                row['person_tmdbId'],
                row['\ufeffmovieId']
            )

print("Relationships created successfully.")
driver.close()

Relationships created successfully.
